In [1]:
import pandas as pd
import numpy as np

In [2]:
all_user = pd.read_csv("data_identification.csv")
tweets = pd.read_csv("tweet_.csv", lineterminator='\n')
tweets = tweets.drop(columns=['Unnamed: 0'], axis=1)
print(all_user.shape)
print(tweets.shape)
tweets.head()

all_user = pd.merge(all_user,tweets, on="tweet_id")
all_user.head()

(1867535, 2)
(1867535, 4)


,tweet_id,identification,_score,hashtags,text
0,0x28cc61,test,107,[],@Habbo I've seen two separate colours of the e...
1,0x29e452,train,809,[],Huge Respect🖒 @JohnnyVegasReal talking about l...
2,0x2b3819,train,808,"['spateradio', 'app']",Yoooo we hit all our monthly goals with the ne...
3,0x2db41f,test,728,[],@FoxNews @KellyannePolls No serious self respe...
4,0x2a2acc,train,16,[],@KIDSNTS @PICU_BCH @uhbcomms @BWCHBoss Well do...


In [3]:
fliter = (all_user["identification"] == 'train')
df_train = all_user[fliter]
fliter1 = (all_user["identification"] == 'test')
df_test = all_user[fliter1]
print(df_train.head(), df_train.shape)
print(df_test.head(), df_test.shape)

   tweet_id identification  _score  \
1  0x29e452          train     809   
2  0x2b3819          train     808   
4  0x2a2acc          train      16   
5  0x2a8830          train     768   
6  0x20b21d          train      70   

                                            hashtags  \
1                                                 []   
2                              ['spateradio', 'app']   
4                                                 []   
5  ['PUBG', 'GamersUnite', 'twitch', 'BeHealthy',...   
6                       ['strength', 'bones', 'God']   

                                                text  
1  Huge Respect🖒 @JohnnyVegasReal talking about l...  
2  Yoooo we hit all our monthly goals with the ne...  
4  @KIDSNTS @PICU_BCH @uhbcomms @BWCHBoss Well do...  
5  Come join @ambushman27 on #PUBG while he striv...  
6  @fanshixieen2014 Blessings!My #strength little...   (1455563, 5)
    tweet_id identification  _score            hashtags  \
0   0x28cc61           test     

In [4]:
emotion = pd.read_csv("emotion.csv")
df_train = pd.merge(df_train,emotion, on="tweet_id")
df_train.iloc[4:100]

df_train = df_train.drop(columns=['identification'],axis=1)
df_test = df_test.drop(columns=['identification'],axis=1)

In [5]:
import string

list_hash = df_train['hashtags'].tolist()
for i in range(len(list_hash)):
    if len(list_hash[i]) == 0:
        list_hash[i]=None
    else:
        list_hash[i]=list_hash[i][1:len(list_hash[i])-2]
        list_hash[i] = [word.strip(string.punctuation) for word in list_hash[i].split(", ")]
        list_hash[i] = ' '.join(list_hash[i])
df_train['hashtags'] = list_hash
df_train["Full_text"] = df_train["hashtags"] + " " + df_train["text"]
df_train.head()

list_hash = df_test['hashtags'].tolist()
for i in range(len(list_hash)):
    if len(list_hash[i]) == 0:
        list_hash[i]=None
    else:
        list_hash[i]=list_hash[i][1:len(list_hash[i])-2]
        list_hash[i] = [word.strip(string.punctuation) for word in list_hash[i].split(", ")]
        list_hash[i] = ' '.join(list_hash[i])
df_test['hashtags'] = list_hash
df_test["Full_text"] = df_test["hashtags"] + " " + df_test["text"]
df_test.head()


,tweet_id,_score,hashtags,text,Full_text
0,0x28cc61,107,,@Habbo I've seen two separate colours of the e...,@Habbo I've seen two separate colours of the ...
3,0x2db41f,728,,@FoxNews @KellyannePolls No serious self respe...,@FoxNews @KellyannePolls No serious self resp...
15,0x2466f6,491,womendrivers,"Looking for a new car, and it says 1 lady owne...","womendrivers Looking for a new car, and it say..."
23,0x23f9e9,28,robbingmembers,@cineworld “only the brave” just out and fount...,robbingmembers @cineworld “only the brave” jus...
31,0x1fb4e1,925,,Felt like total dog 💩 going into open gym and ...,Felt like total dog 💩 going into open gym and...


In [6]:
from sklearn.utils import shuffle
df_train = shuffle(df_train)  

In [12]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

tfIdfVectorizer_1000 = TfidfVectorizer(max_features = 6000, tokenizer=nltk.word_tokenize, use_idf=True)

tfIdfVectorizer_1000.fit(df_train['Full_text'])

t = tfIdfVectorizer_1000.transform(df_train['Full_text'])
t.shape

/opt/conda/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


(1455563, 3)

In [7]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

tfIdfVectorizer_1 = TfidfVectorizer(max_features = 5500, tokenizer=nltk.word_tokenize, use_idf=True, max_df=0.7)
tfIdfVectorizer_2 = TfidfVectorizer(max_features = 1000, tokenizer=nltk.word_tokenize, use_idf=True)

# tf_train_data = pd.concat([df_train['hashtags'], df_train['text']], axis=1)
# df_train['ti_hash'] = tfIdfVectorizer.fit_transform(df_train['hashtags']).toarray()
# df_train['ti_text'] = tfIdfVectorizer.fit_transform(df_train['text']).toarray()
tfIdfVectorizer_1.fit(df_train['text'])
tfIdfVectorizer_2.fit(df_train['hashtags'])

t1 = tfIdfVectorizer_1.transform(df_train['text'])
t2 = tfIdfVectorizer_2.transform(df_train['hashtags'])
print(t1.shape, t2.shape)
t = np.hstack((t1.toarray(),t2.toarray()))
t.shape

/opt/conda/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
/opt/conda/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


(1455563, 5500) (1455563, 1000)


(1455563, 6500)

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    t,
    df_train['emotion'], test_size=0.0001)

In [26]:
## deal with label (string -> one-hot)
import keras
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)
y_test = label_encode(label_encoder, y_test)

print('\n\n## After convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)



check label:  ['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']

## Before convert
y_train[0:4]:
 240834    disgust
893058        joy
588584        joy
519950    sadness
Name: emotion, dtype: object

y_train.shape:  (1455417,)
y_test.shape:  (146,)


## After convert
y_train[0:4]:
 [[0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]]

y_train.shape:  (1455417, 8)
y_test.shape:  (146, 8)


In [27]:
# I/O check
input_shape = X_train.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

input_shape:  6500
output_shape:  8


In [34]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax
import tensorflow as tf

# input layer
model_input = Input(shape=(input_shape, ))  # 500
X = model_input

# 1st hidden layer
X_W0 = Dense(units=2048)(X)  # 64
H0 = ReLU()(X_W0)

X_W1 = Dense(units=1024)(H0)  # 64
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=512)(H1)  # 64
H2 = ReLU()(H1_W2)

# output layer
H2_W3 = Dense(units=256)(H2)  # 4
H3 = ReLU()(H2_W3)

H3_W4 = Dense(units=128)(H3)  # 4
H4 = ReLU()(H3_W4)

H4_W5 = Dense(units=64)(H4)  # 4
H5 = ReLU()(H4_W5)

H5_W6 = Dense(units=output_shape)(H5)  # 4
H6 = Softmax()(H5_W6)

model_output = H6

# create model
model = Model(inputs=[model_input], outputs=[model_output])

loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4)

# loss function & optimizer
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 6500)]            0         
_________________________________________________________________
dense_42 (Dense)             (None, 2048)              13314048  
_________________________________________________________________
re_lu_36 (ReLU)              (None, 2048)              0         
_________________________________________________________________
dense_43 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
re_lu_37 (ReLU)              (None, 1024)              0         
_________________________________________________________________
dense_44 (Dense)             (None, 512)               524800    
_________________________________________________________________
re_lu_38 (ReLU)              (None, 512)               0   

In [35]:
from keras.callbacks import CSVLogger

# X_train = X_train.todense()
# X_test = X_test.todense()


# training setting
epochs = 5
batch_size = 32

# training!
history = model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    validation_data = (X_test, y_test))
print('training finish')

Epoch 1/5
45482/45482 [==============================] - 1581s 35ms/step - loss: 1.2193 - accuracy: 0.5598 - val_loss: 1.2825 - val_accuracy: 0.5205
Epoch 2/5
45482/45482 [==============================] - 1637s 36ms/step - loss: 1.0617 - accuracy: 0.6198 - val_loss: 1.2652 - val_accuracy: 0.5685
Epoch 3/5
45482/45482 [==============================] - 1658s 36ms/step - loss: 0.8393 - accuracy: 0.7056 - val_loss: 1.2848 - val_accuracy: 0.5753
Epoch 4/5
45482/45482 [==============================] - 1661s 37ms/step - loss: 0.5914 - accuracy: 0.7971 - val_loss: 1.4900 - val_accuracy: 0.5685
Epoch 5/5
45482/45482 [==============================] - 1657s 36ms/step - loss: 0.4158 - accuracy: 0.8599 - val_loss: 1.6946 - val_accuracy: 0.5411
training finish


In [83]:
## predict
X_ans = tfIdfVectorizer_1000.transform(df_test["Full_text"])


In [20]:
t1 = tfIdfVectorizer_1.transform(df_test['text'])
t2 = tfIdfVectorizer_2.transform(df_test['hashtags'])
X_ans = np.hstack((t1.toarray(),t2.toarray()))

In [36]:
# X_ans = X_ans.todense()
pred_result = model.predict(X_ans, batch_size=128)
pred_result[:5]

array([[1.97626092e-03, 7.22786970e-03, 1.35586075e-02, 3.48014222e-03,
        4.73394804e-02, 3.02087050e-03, 6.70580268e-01, 2.52816528e-01],
       [1.01433034e-06, 3.72975919e-05, 1.48794579e-03, 1.19729348e-05,
        3.33004573e-05, 9.98312473e-01, 1.11889945e-04, 4.17271031e-06],
       [2.34003905e-02, 2.13227095e-03, 3.55986297e-01, 2.80465167e-02,
        2.76304707e-02, 5.27450860e-01, 1.61983985e-02, 1.91547684e-02],
       [2.65090307e-03, 6.46696790e-05, 9.95756805e-01, 3.12246964e-04,
        2.70715536e-04, 5.14465384e-04, 3.88071378e-04, 4.20986253e-05],
       [8.93621027e-01, 1.78704315e-04, 1.40280686e-02, 1.20199739e-03,
        7.47758150e-02, 1.06750252e-02, 2.79533397e-03, 2.72411155e-03]],
      dtype=float32)

In [37]:
pred_result = label_decode(label_encoder, pred_result)
pred_result[:5]

array(['surprise', 'sadness', 'sadness', 'disgust', 'anger'], dtype=object)

In [38]:
df_test['emotion'] = pred_result
df_answer = pd.concat([df_test['tweet_id'],
                     df_test['emotion']], axis=1)
df_answer

,tweet_id,emotion
0,0x28cc61,surprise
3,0x2db41f,sadness
15,0x2466f6,sadness
23,0x23f9e9,disgust
31,0x1fb4e1,anger
...,...,...
1867495,0x2c4dc2,joy
1867496,0x31be7c,anticipation
1867500,0x1ca58e,trust
1867515,0x35c8ba,sadness


In [39]:
df_answer = df_answer.rename(columns={'tweet_id':'id'})
df_answer.to_csv('ans.csv',encoding='utf-8', index=False)